In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://securit

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

## Count numbere of rows

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

3093869
3093660
3093660


Transform to fit schema

In [6]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100J4AOWOWCTX|   43188629|B00CP1N45K|     556368387| 2013-10-14|
|R100N7Y79EQNCL|   41773257|B008VMT2HQ|     178314766| 2015-06-30|
|R1015SVKAKV7EQ|   37180271|B00CL8FFW2|     643672885| 2014-06-19|
|R101BIRNEZ17XW|   22896030|B005C1KG9M|     205202992| 2012-12-24|
|R102PG7ZKS5IFS|   40119038|B00413ZCVO|     195242426| 2011-10-03|
|R10398PM75STU0|   50649955|B000AHVGC4|     487899844| 2010-08-30|
|R103QXL0PLKBVO|   15650406|B00316263Y|     945305640| 2011-07-12|
|R103Z5DGS3VBRK|   22001706|B00BLZ7EYI|     212214930| 2014-02-27|
|R104271C2Q2DCM|    2468415|B0035JKJ2W|     963937571| 2013-12-09|
|R1049GRPORPPOG|   51117624|B005UNPM3M|        497843| 2015-05-28|
|R104OZXSZEA872|   21864230|B00067OF80|      12310937| 2008-09-25|
|R104XNJ9PUIYBW|   40874429|B00H45LDFU|     351908409| 2015-07

In [8]:
product_table = df.select(["product_id", "product_title"])
product_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CP1N45K|ViewHD Advanced H...|
|B008VMT2HQ|TDK Life on Recor...|
|B00CL8FFW2|GE 87670 RG6 Crim...|
|B005C1KG9M|Replacment Ear Pa...|
|B00413ZCVO|Pioneer Elite VSX...|
|B000AHVGC4|APC AV J10 Home T...|
|B00316263Y|BlueRigger Basic ...|
|B00BLZ7EYI|uxcell DC 5V 1-Ch...|
|B0035JKJ2W|Sennheiser Earfin...|
|B005UNPM3M|Titanium Smart Fa...|
|B00067OF80|Sennheiser EH-150...|
|B00H45LDFU|Elite Screens DIY...|
|B002M3SO0G|Apple 16 GB iPod ...|
|B0002ZPF6W|Parts Express 2.5...|
|B004QK7HI8|Mohu Leaf 30 TV A...|
|B009EIUH6G|Logitech Harmony ...|
|B00BBDKTJC|Audio Technica AT...|
|B008MG5V80|ECOXGEAR ECOXBT R...|
|B00004SD88|Sennheiser Open D...|
|B000BMQPCE|Premier Mounts FL...|
+----------+--------------------+
only showing top 20 rows



In [10]:
customers_table = df.groupby("customer_id").agg({"customer_id":"count"})
customers= customers_table.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   41626553|             1|
|   46134234|             4|
|   13705152|             1|
|   44413267|             1|
|   36644422|             2|
|    8061365|             1|
|   49196304|             9|
|   11587470|             1|
|   25538476|             3|
|   46889381|             3|
|   11263585|             6|
|   12865675|             5|
|   51398878|             1|
|   46791882|             1|
|   52689018|             1|
|   51185108|             3|
|   25750713|             1|
|   49696648|             2|
|   25425404|             1|
|   33093303|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

In [11]:
mode="append"

jdbc_url="jdbc:postgresql://database-1.cv1pxdhixoh6.us-west-2.rds.amazonaws.com:5432/database-1"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [12]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)


In [ ]:
product_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)